## Working with an LLM programmatically

You have certainly interacted before with a Large Language Model (LLM) like ChatGPT. This is usually done through a UI or an application.

In this Notebook, we are going to use Python to connect and query an LLM directly through its API. For this Lab we have selected the model **OLLAMA Mistral-7B**.[(https://ollama.com/library/mistral](https://ollama.com/library/mistral)). This is a fully Open Source model (Apache 2.0 license), that although much lighter than other commercial or open source models is very capable, especially with the tasks we intend to use it for.

This model has already been deployed on the Lab cluster because even if it's a smaller model we are hosting it on CPUs instead of GPUs which slows down the LLM service but provides us with low cost option for a demo environment. Ideally you will need a GPU with atleast 24GB RAM.

In [1]:
!pip install -q langchain==0.1.14


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


### Requirements and Imports

If you have selected the right workbench image to launch as per the Lab's instructions, you should already have all the needed libraries. If not uncomment the first line in the next cell to install all the right packages. We will then import the libraries we need.

In [2]:
# Imports
import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import Ollama
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

### Langchain

Langchain (https://www.langchain.com/) is a framework for developing applications powered by language models. It will take care for us of all the boilerplate code we would have to manually write to properly query an LLM.

We will start by creating an **llm** instance, defined by the location where the LLM API can be queried and some parameters that will be applied to the model. For example, `max_new_tokens` will instruct the model to answer with a maximum of 96 tokens (words or parts of words). `temperature`, set really low here, will instruct the model to stay truth-grounded, and not try to be too "creative". After all, we're not trying to write a fancy poem here!

In [3]:
inference_server_url = "http://llm.ic-shared-llm.svc.cluster.local:11434"

In [4]:
# LLM definition
llm = Ollama(
    base_url=inference_server_url,
    model="mistral",
    top_p=0.92,
    temperature=0.01,
    num_predict=512,
    repeat_penalty=1.03,
    callbacks=[StreamingStdOutCallbackHandler()]
)

print(llm)

Ollama
Params: {'model': 'mistral', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': 512, 'repeat_last_n': None, 'repeat_penalty': 1.03, 'temperature': 0.01, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': 0.92}, 'system': None, 'template': None, 'keep_alive': None}


We also need a **template** to be applied to every request we are sending to the model (the "Prompt").

When querying a model, you almost never want to send directly what the user has typed. On top of this entry, you need to give proper instructions to the model so that it knows how to handle it: what and how to answer, what NOT to answer, the tone it must use...

In [5]:
template="""<s>[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe.
You will be asked a question, to which you must give an answer.
Your answer should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, answer "I don't know".
<</SYS>>

### QUESTION:
{input}

### ANSWER:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["input"], template=template)

Langchain allows us to now easily "stitch" those elements together and create a **conversation** object that we will use to query the model.

In [6]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )

We are now ready to query the model! Wait for the output from the query. It will take a minute or two since we are using cpu instead of gpu for LLM service.

In [7]:
query = "What is Artificial Intelligence?"
conversation.predict(input=query); # ";" at the end of the line hides final output (repetion of the streamed answer)

 Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. Rather than being limited by their pre-programmed instructions, AI systems are designed to adapt and improve from experience. They can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI systems can be categorized as either narrow or general. Narrow AI is designed to perform a specific task, such as playing chess or recognizing speech, while general AI, also known as artificial general intelligence, has the ability to understand, learn, and apply knowledge across a wide range of tasks at a level equal to or beyond a human being.